Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Install and Import Libraries
Let's install PySpark:

## Build Spark Session

In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as Fns

spark = SparkSession.builder.appName('practical_1').getOrCreate()


## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [2]:
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)


Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [3]:
print(f"Train type: {type(train_df)} \nTest type: {type(test_df)}")

Train type: <class 'pyspark.sql.dataframe.DataFrame'> 
Test type: <class 'pyspark.sql.dataframe.DataFrame'>


**Show 5 rows.**

In [4]:
train_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [5]:
train_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [6]:
train_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [7]:
train_df.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [8]:
survived_count = train_df.filter(Fns.col('Survived') == 1).count()
not_survived_count = train_df.filter(Fns.col('Survived') == 0).count()

**Display your result:**

In [9]:
train_df.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [10]:
print(f"Survived Count: {survived_count} | Not Survived Count: {not_survived_count}")

Survived Count: 342 | Not Survived Count: 549


**Can you display your answer in ratio form?(Hint: Use "UDF" Function. (Hint: Use "UDF" Function. This is a hint you can use any method.)**






In [11]:
# survive_ratio_udf = Fns.udf(lambda x: x.filter(
#     Fns.col('Survived') == 1).count() / x.filter(Fns.col('Survived') == 0).count())

# survive_ratio_udf= lambda x: x.filter(
#     Fns.col('Survived') == 1) / x.filter(Fns.col('Survived') == 0)

print(f"Survive Ratio: {survived_count / not_survived_count}")


Survive Ratio: 0.6229508196721312


**Can you get the number of males and females?**


In [12]:
train_df.groupBy('Sex').count().show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column. This is a hint you can use any method.)

In [13]:
# Average of `Survived` grouped by `Sex`
train_df.groupby('Sex').mean().select(Fns.col('avg(Survived)')).show()

# Number of `Survived` grouped by `Sex`
train_df.filter(Fns.col("Survived") == 1).groupby(['Sex']).count().show()

+-------------------+
|      avg(Survived)|
+-------------------+
| 0.7420382165605095|
|0.18890814558058924|
+-------------------+

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



**Create temporary view PySpark:**

In [14]:
train_df.createOrReplaceTempView('train_tmp_view')

**How many people survived, and how many didn't survive? By SQL:**

In [15]:
spark.sql('SELECT Survived, COUNT(*) FROM train_tmp_view GROUP BY Survived').show()

+--------+--------+
|Survived|count(1)|
+--------+--------+
|       1|     342|
|       0|     549|
+--------+--------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column. This is a hint you can use any method.)

**Can you do this via SQL?**

In [16]:
spark.sql('\
        SELECT Sex, COUNT(*) as GenderSurviveCount, COUNT(*) * 100.0 / SUM(COUNT(*)) over () as GenderSurviveRatio \
        FROM train_tmp_view \
        WHERE Survived=1 \
        GROUP BY Sex \
    ').show()


+------+------------------+------------------+
|   Sex|GenderSurviveCount|GenderSurviveRatio|
+------+------------------+------------------+
|female|               233| 68.12865497076023|
|  male|               109| 31.87134502923977|
+------+------------------+------------------+



**Display a ratio for "p-class": SUM(Survived)/count for p-class**


In [17]:
spark.sql('\
        SELECT \
            Pclass, \
            SUM(Survived) as TotalSurvive, \
            COUNT(*) as PClassSurviveRatio, \
            SUM(Survived) / COUNT(*) as PClassSurviveRatio \
        FROM train_tmp_view \
        GROUP BY Pclass \
    ').show()


+------+------------+------------------+-------------------+
|Pclass|TotalSurvive|PClassSurviveRatio| PClassSurviveRatio|
+------+------------+------------------+-------------------+
|     1|         136|               216| 0.6296296296296297|
|     3|         119|               491|0.24236252545824846|
|     2|          87|               184|0.47282608695652173|
+------+------------+------------------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [18]:
both_df = train_df.unionAll(test_df)

**Display count:**

In [19]:
both_df.count()

1329

**Can you define the number of null values in each column?**


In [20]:
# both_df.select([Fns.count(Fns.when(Fns.isnan(c), c)).alias(c) for c in both_df.columns]).show()
na_dict = {col: both_df.filter(both_df[col].isNull()).count() for col in both_df.columns}
na_dict

{'PassengerId': 0,
 'Survived': 0,
 'Pclass': 0,
 'Name': 0,
 'Sex': 0,
 'Age': 265,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': 0,
 'Fare': 0,
 'Cabin': 1021,
 'Embarked': 3}

**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [21]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

missing_vals_df = spark.createDataFrame(na_dict.items(), schema=StructType([
    StructField("Column", StringType(), True),
    StructField("NumberOfMissingValues", IntegerType(), True)
]))
print(missing_vals_df.schema)
missing_vals_df.show()


StructType(List(StructField(Column,StringType,true),StructField(NumberOfMissingValues,IntegerType,true)))
+-----------+---------------------+
|     Column|NumberOfMissingValues|
+-----------+---------------------+
|PassengerId|                    0|
|   Survived|                    0|
|     Pclass|                    0|
|       Name|                    0|
|        Sex|                    0|
|        Age|                  265|
|      SibSp|                    0|
|      Parch|                    0|
|     Ticket|                    0|
|       Fare|                    0|
|      Cabin|                 1021|
|   Embarked|                    3|
+-----------+---------------------+



## Preprocessing 

**Create Temporary view PySpark:**

In [22]:
both_df.createOrReplaceTempView('both_tmp_view')

**Can you show the "name" column from your temporary table?**

In [23]:
spark.sql('SELECT name FROM both_tmp_view').show(5)

+--------------------+
|                name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
+--------------------+
only showing top 5 rows



**Run this code:**

In [24]:
both_df = both_df.withColumn('Title',Fns.regexp_extract(Fns.col("Name"),"([A-Za-z]+)\.",1))
both_df.createOrReplaceTempView('both_tmp_view')

**Display "Title" column and count "Title" column:**

In [25]:
spark.sql('SELECT Title, COUNT(Title) FROM both_tmp_view GROUP BY Title').show()

+--------+------------+
|   Title|count(Title)|
+--------+------------+
|     Don|           1|
|    Miss|         257|
|Countess|           2|
|     Col|           4|
|     Rev|           9|
|    Lady|           2|
|  Master|          56|
|     Mme|           1|
|    Capt|           2|
|      Mr|         786|
|      Dr|          11|
|     Mrs|         186|
|     Sir|           2|
|Jonkheer|           2|
|    Mlle|           4|
|   Major|           3|
|      Ms|           1|
+--------+------------+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [26]:
titles_map = {
    'Dr': 'rare', 'Rev': 'rare', 'Major': 'rare', 
    'Col': 'rare', 'Mlle': 'rare', 'Capt': 'rare', 
    'Don': 'rare', 'Jonkheer': 'rare', 'Countess': 'rare', 
    'Ms': 'rare', 'Sir': 'rare', 'Lady': 'rare', 
    'Mme': 'rare', 
    'Mr': 'Mr', 'Mrs': 'Mrs', 'Master': 'Master', 'Miss': 'Miss'
}
titles_map

{'Dr': 'rare',
 'Rev': 'rare',
 'Major': 'rare',
 'Col': 'rare',
 'Mlle': 'rare',
 'Capt': 'rare',
 'Don': 'rare',
 'Jonkheer': 'rare',
 'Countess': 'rare',
 'Ms': 'rare',
 'Sir': 'rare',
 'Lady': 'rare',
 'Mme': 'rare',
 'Mr': 'Mr',
 'Mrs': 'Mrs',
 'Master': 'Master',
 'Miss': 'Miss'}

**Run the function:**

In [27]:
def impute_title(title):
    return titles_map[title]# Title_map is your dictionary. please change this name with your dictionary name.

**Apply the function on "Title" column using UDF:**

In [28]:
impute_title_UDF = Fns.udf(lambda x: impute_title(x))
both_df = both_df.withColumn('Title', impute_title_UDF(Fns.col('Title')))

**Display "Title" from table and group by "Title" column:**

In [29]:
both_df.createOrReplaceTempView('both_tmp_view')
spark.sql('SELECT Title FROM both_tmp_view GROUP BY Title').show()

+------+
| Title|
+------+
|  rare|
|  Miss|
|Master|
|    Mr|
|   Mrs|
+------+



## **Preprocessing Age**

**Based on the "age" column mean, you will fill in the missing age values:**

In [36]:
from pyspark.ml.feature import Imputer

AgeImputer = Imputer(
    strategy='mean',
    inputCols = ['Age'],
    outputCols = ['Age']
)


**Fill missing with "age" mean:**

In [37]:
both_df = AgeImputer.fit(both_df).transform(both_df)
print(f"Age Missing Value Count: {both_df.filter(both_df['Age'].isNull()).count()}")

Age Missing Value Count: 0


## **Preprocessing Embarked**

**Select "Embarked" column, count them, order by count Desc, and save in grouped_Embarked variable:**




In [38]:
both_df.createOrReplaceTempView('both_tmp_view')
grouped_Embarked = spark.sql('\
        SELECT Embarked, COUNT(*) as EmbarkedCount\
        FROM both_tmp_view \
        GROUP BY Embarked \
        ORDER BY COUNT(*) DESC\
    ')


**Show "groupped_Embarked" your variable:**

In [39]:
grouped_Embarked.show()


+--------+-------------+
|Embarked|EmbarkedCount|
+--------+-------------+
|       S|          965|
|       C|          253|
|       Q|          111|
+--------+-------------+



**Get max of groupped_Embarked:** 

In [40]:
grouped_Embarked.first()

Row(Embarked='S', EmbarkedCount=965)

**Fill missing values with max 'S' of grouped_Embarked:**

In [42]:
both_df = both_df.fillna({'Embarked':grouped_Embarked.first()['Embarked']})
both_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|  Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|   Mr|
+-----------+--------+------+---

## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



In [44]:
both_df = both_df.withColumn("Cabin", Fns.substring('Cabin', 0, 1))


**Show the result:**

In [45]:
both_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S| Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    C|       S|  Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|   Mr|
+-----------+--------+------+---

**Create the temporary view:**

In [46]:
both_df.createOrReplaceTempView('both_tmp_view')

**Select "Cabin" column, count "Cabin" column, Group by "Cabin" column, Order By count DESC**  

In [49]:
spark.sql('SELECT Cabin, COUNT(*) FROM both_tmp_view GROUP BY Cabin ORDER BY COUNT(*) DESC').show()

+-----+--------+
|Cabin|count(1)|
+-----+--------+
| null|    1021|
|    C|      82|
|    B|      77|
|    D|      52|
|    E|      51|
|    A|      23|
|    F|      18|
|    G|       4|
|    T|       1|
+-----+--------+



**Fill missing values with "U":**

In [51]:
both_df = both_df.fillna({'Cabin':'U'})
both_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Title|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25|    U|       S|   Mr|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|    C|       C|  Mrs|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925|    U|       S| Miss|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1|    C|       S|  Mrs|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05|    U|       S|   Mr|
+-----------+--------+------+---

**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

In [95]:
both_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = false)
 |-- Embarked: string (nullable = false)
 |-- Title: string (nullable = true)



In [112]:
to_drop_cols = ['PassengerId', 'Name', 'Ticket']
cat_cols = ['Sex', 'Cabin', 'Embarked', 'Title']


In [113]:
from pyspark.ml.feature import StringIndexer

# Convert string to number using StringIndexer
category_indexers = []
for col in cat_cols:
    category_indexers.append(StringIndexer(inputCol=col, outputCol=f"{col}_index"))
    # Fits a model to the input dataset with optional parameters.
    # both_df = category_indexer.fit(both_df).transform(both_df)

# both_df.show(5)

**OneHotEncoder(inputCols=None, outputCols=None)**

A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index. For example with 5 categories, an input value of 2.0 would map to an output vector of [0.0, 0.0, 1.0, 0.0]. The last category is not included by default (configurable via dropLast), because it makes the vector entries sum up to one, and hence linearly dependent. So an input value of 4.0 maps to [0.0, 0.0, 0.0, 0.0].

In [114]:
from pyspark.ml.feature import OneHotEncoder

# OneHot Encoding
oh_encoders = []
for col in cat_cols:
    oh_encoders.append(OneHotEncoder(inputCol=f"{col}_index", outputCol=f"{col}_oh_encoded"))
    # Fits a model to the input dataset with optional parameters.
    # both_df = oh_encoders.fit(both_df).transform(both_df)

# both_df.show(5)

In [120]:
# Create list for features names
features_list = list(set(both_df.columns) - (set(cat_cols + to_drop_cols + ['Survived'])))
for col in cat_cols:
    features_list.append(f"{col}_oh_encoded")

features_list


['Pclass',
 'Parch',
 'Age',
 'Fare',
 'SibSp',
 'Sex_oh_encoded',
 'Cabin_oh_encoded',
 'Embarked_oh_encoded',
 'Title_oh_encoded']

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None). A feature transformer that merges multiple columns into a vector column.**



In [121]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols = features_list,
    outputCol = "features"
)


**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [122]:
# Split data into train & test
(x_train, x_test) = both_df.randomSplit([.8, .2])


**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [123]:
from pyspark.ml.classification import RandomForestClassifier

rfc = RandomForestClassifier(labelCol='Survived', featuresCol='features')


In [124]:
from pyspark.ml import Pipeline

# create pipeline
stages = category_indexers + oh_encoders + [assembler] + [rfc]
pipeline = Pipeline(stages=stages)

# train RFC model
pipeline_model = pipeline.fit(x_train)

# get train predictions
train_predictions = pipeline_model.transform(x_train)
train_predictions.select('features', 'Survived', 'prediction').show(3)

# get test predictions 
test_predictions = pipeline_model.transform(x_test)


+--------------------+--------+----------+
|            features|Survived|prediction|
+--------------------+--------+----------+
|(20,[0,2,3,4,5,6,...|       0|       0.0|
|(20,[0,2,3,4,7,15...|       1|       1.0|
|(20,[0,2,3,6,14,1...|       1|       1.0|
+--------------------+--------+----------+
only showing top 3 rows



**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [125]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rfc_eval = MulticlassClassificationEvaluator(
    labelCol="Survived", predictionCol="prediction", metricName="accuracy")

# training Accuracy
print("Training Accuracy: %.3f" % rfc_eval.evaluate(train_predictions))

# testing Accuracy
print("Testing Accuracy: %.3f" % rfc_eval.evaluate(test_predictions))


Training Accuracy: 0.846
Testing Accuracy: 0.849
